In [14]:
import numpy as np
import pandas as pd

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten, Permute, Reshape
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.regularizers import l2
from keras.utils.vis_utils import plot_model

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

import cv2

In [2]:
train_path = 'Data/Train'
valid_path = 'Data/Validate'
#test_path = 'Data/Test'

In [3]:
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(128,128), classes=['Sample001','Sample002','Sample003','Sample004','Sample005','Sample006','Sample007','Sample008','Sample009','Sample010','Sample011','Sample012','Sample013','Sample014','Sample015','Sample016','Sample017','Sample018','Sample019','Sample020','Sample021','Sample022','Sample023','Sample024','Sample025','Sample026','Sample027','Sample028','Sample029','Sample030','Sample031','Sample032','Sample033','Sample034','Sample035','Sample036'], batch_size=15)

Found 36576 images belonging to 36 classes.


In [4]:
valid_batches= ImageDataGenerator().flow_from_directory(valid_path, target_size=(128,128), classes=['Sample001','Sample002','Sample003','Sample004','Sample005','Sample006','Sample007','Sample008','Sample009','Sample010','Sample011','Sample012','Sample013','Sample014','Sample015','Sample016','Sample017','Sample018','Sample019','Sample020','Sample021','Sample022','Sample023','Sample024','Sample025','Sample026','Sample027','Sample028','Sample029','Sample030','Sample031','Sample032','Sample033','Sample034','Sample035','Sample036'], batch_size=15)

Found 1980 images belonging to 36 classes.


In [5]:
#test_batches= ImageDataGenerator().flow_from_directory(test_path, target_size=(128,128), classes=['Sample001','Sample002','Sample003','Sample004','Sample005','Sample006','Sample007','Sample008','Sample009','Sample010','Sample011','Sample012','Sample013','Sample014','Sample015','Sample016','Sample017','Sample018','Sample019','Sample020','Sample021','Sample022','Sample023','Sample024','Sample025','Sample026','Sample027','Sample028','Sample029','Sample030','Sample031','Sample032','Sample033','Sample034','Sample035','Sample036'], batch_size=15)

In [6]:
weight_decay = 0.001
input_shape=(128,128,3)
batchSize=8
num_classes=36
epochs=10

model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), W_regularizer=l2(weight_decay), padding="same", activation="relu", input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(128, kernel_size=(3,3), W_regularizer=l2(weight_decay), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(256, kernel_size=(3,3), W_regularizer=l2(weight_decay), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(256, kernel_size=(3,3), W_regularizer=l2(weight_decay), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(512, kernel_size=(3,3), W_regularizer=l2(weight_decay), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(512, kernel_size=(3,3), W_regularizer=l2(weight_decay),padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(512, kernel_size=(3,3), W_regularizer=l2(weight_decay), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # (bs, y, x, c) --> (bs, x, y, c)
model.add(Permute((2, 1, 3)))

    # (bs, x, y, c) --> (bs, x, y * c)
bs, x, y, c = model.layers[-1].output_shape
model.add(Reshape((x, y*c)))

model.add(Bidirectional(LSTM(256, return_sequences=False), merge_mode="concat"))
model.add(Dense(num_classes, activation="softmax"))

/home/ssatyanarayana/.virtualenvs/virtual-vision/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu", input_shape=(128, 128,..., kernel_regularizer=<keras.reg...)`
  if __name__ == '__main__':
/home/ssatyanarayana/.virtualenvs/virtual-vision/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, kernel_size=(3, 3), padding="same", activation="relu", kernel_regularizer=<keras.reg...)`
  del sys.path[0]
/home/ssatyanarayana/.virtualenvs/virtual-vision/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, kernel_size=(3, 3), padding="same", activation="relu", kernel_regularizer=<keras.reg...)`
/home/ssatyanarayana/.virtualenvs/virtual-vision/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 128)       512       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 256)       295168    
__________

In [8]:
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [9]:
OcrData=model.fit_generator(train_batches,epochs=epochs,verbose=1,validation_data=valid_batches)

Epoch 1/10
2439/2439 [==============================] - 214s 88ms/step - loss: 1.1532 - acc: 0.8793 - val_loss: 2.9873 - val_acc: 0.3758
Epoch 2/10
2439/2439 [==============================] - 144s 59ms/step - loss: 0.6594 - acc: 0.9261 - val_loss: 3.9794 - val_acc: 0.2596
Epoch 3/10
2439/2439 [==============================] - 143s 59ms/step - loss: 0.6245 - acc: 0.9304 - val_loss: 2.9737 - val_acc: 0.3965
Epoch 4/10
2439/2439 [==============================] - 144s 59ms/step - loss: 0.5843 - acc: 0.9345 - val_loss: 2.8908 - val_acc: 0.3838
Epoch 5/10
2439/2439 [==============================] - 145s 59ms/step - loss: 0.5458 - acc: 0.9394 - val_loss: 2.8963 - val_acc: 0.4369
Epoch 6/10
2439/2439 [==============================] - 144s 59ms/step - loss: 0.4899 - acc: 0.9450 - val_loss: 2.5064 - val_acc: 0.4838
Epoch 7/10
2439/2439 [==============================] - 145s 59ms/step - loss: 0.4475 - acc: 0.9477 - val_loss: 2.8324 - val_acc: 0.4621
Epoch 8/10
2439/2439 [===================

In [10]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [11]:
test_image=cv2.imread(args["ocr/2.png"])

In [28]:
import time

In [62]:
img = cv2.imread('ocr/8.png', 1)

img = img.copy()
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

r = 128 / img.shape[1]
dim = (128, 128)
 
# perform the actual resizing of the image and show it
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

detections = model.predict_on_batch(np.expand_dims(resized, axis=0))
print (detections)

[[0.03957945 0.06310543 0.0125579  0.00539502 0.00125132 0.00215507
  0.01195587 0.01389483 0.00987881 0.04238543 0.01168872 0.00303054
  0.00429773 0.0121133  0.00116464 0.00141445 0.05723098 0.00473035
  0.22970699 0.33564103 0.01079115 0.02071884 0.00646836 0.00719469
  0.01997122 0.00557953 0.01700069 0.00633833 0.00792162 0.00678159
  0.00264783 0.00399406 0.00964669 0.00220966 0.00351161 0.00604621]]


In [63]:
np.max(detections)

0.33564103

In [64]:
detections.argmax()


19